#### Creating a class


In [ ]:
class wikipedia_webscrap:
  def __init__(self):
    ''' The function reads the website, extracts the table from it and store it
    as a pandas DataFrame'''

    import pandas as pd
    # Read data from the specified website
    self.wiki_page=pd.read_html('https://en.wikipedia.org/wiki/COVID-19_pandemic_by_country_and_territory')
    # Read data from the specified table
    self.covid_table=self.wiki_page[9]


  def drop_columns(self,remove_list,axis=1):
    ''' Pass the list of columns to be dropped to the function 
    and it will drop the same from DataFrame'''
    self.covid_table.drop(remove_list,axis=axis,inplace=True)

  def check_info(self):
    '''Check the datatypes and null values in the dataset'''
    self.covid_table.info()



In [ ]:
wiki=wikipedia_webscrap()
table=wiki.covid_table
wiki.drop_columns(['Country','Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7'])
column_names=['country','deaths_per_million','deaths','recovered']
table.columns=column_names
wiki.check_info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   country             218 non-null    object
 1   deaths_per_million  218 non-null    object
 2   deaths              218 non-null    object
 3   recovered           218 non-null    object
dtypes: object(4)
memory usage: 6.9+ KB


#### Checking for improper value in country column to drop

In [ ]:
table['country'].unique()

array(['World[a]', 'Peru', 'Bulgaria', 'Bosnia and Herzegovina',
       'Hungary', 'Montenegro', 'North Macedonia', 'Georgia',
       'Czech Republic', 'Romania', 'Croatia', 'Slovakia', 'Gibraltar',
       'Brazil', 'San Marino', 'Lithuania', 'Armenia', 'Slovenia',
       'Argentina', 'Colombia', 'Poland', 'United States', 'Belgium',
       'Latvia', 'Moldova', 'Ukraine', 'Paraguay', 'Mexico', 'Italy',
       'French Polynesia', 'United Kingdom', 'Tunisia', 'Russia', 'Chile',
       'Suriname', 'European Union[b]', 'Trinidad and Tobago', 'Greece',
       'Spain', 'Ecuador', 'Portugal', 'Serbia', 'France',
       'Liechtenstein', 'Andorra', 'Bahamas', 'Grenada', 'Uruguay',
       'Bermuda', 'Panama', 'Aruba', 'Kosovo', 'Bolivia', 'Saint Lucia',
       'Iran', 'South Africa', 'Austria', 'Sweden', 'Belize', 'Estonia',
       'Luxembourg', 'Costa Rica', 'Namibia', 'Switzerland', 'Lebanon',
       'Guyana', 'Seychelles', 'Germany', 'British Virgin Islands',
       'Netherlands', 'Jordan', '

The last two rows are to be removed as they do not represent any country or region

In [ ]:
wiki.drop_columns([table.shape[0]-1,table.shape[0]-2],0)

In [ ]:
table['country'].unique()

array(['World[a]', 'Peru', 'Bulgaria', 'Bosnia and Herzegovina',
       'Hungary', 'Montenegro', 'North Macedonia', 'Georgia',
       'Czech Republic', 'Romania', 'Croatia', 'Slovakia', 'Gibraltar',
       'Brazil', 'San Marino', 'Lithuania', 'Armenia', 'Slovenia',
       'Argentina', 'Colombia', 'Poland', 'United States', 'Belgium',
       'Latvia', 'Moldova', 'Ukraine', 'Paraguay', 'Mexico', 'Italy',
       'French Polynesia', 'United Kingdom', 'Tunisia', 'Russia', 'Chile',
       'Suriname', 'European Union[b]', 'Trinidad and Tobago', 'Greece',
       'Spain', 'Ecuador', 'Portugal', 'Serbia', 'France',
       'Liechtenstein', 'Andorra', 'Bahamas', 'Grenada', 'Uruguay',
       'Bermuda', 'Panama', 'Aruba', 'Kosovo', 'Bolivia', 'Saint Lucia',
       'Iran', 'South Africa', 'Austria', 'Sweden', 'Belize', 'Estonia',
       'Luxembourg', 'Costa Rica', 'Namibia', 'Switzerland', 'Lebanon',
       'Guyana', 'Seychelles', 'Germany', 'British Virgin Islands',
       'Netherlands', 'Jordan', '

####  Replacing the '[charachter]' in country

In [ ]:

countries=[]
import re
for country in table['country']:
  if len(re.findall('\[\w\]',country))>0:
    countries.append(re.findall('(\w*)\[',country)[0])
  else:
    countries.append(country)
table['country']=countries

In [ ]:
table.head()

,country,deaths_per_million,deaths,recovered
0,World,686,5406818,281400646
1,Peru,6070,202524,2279299
2,Bulgaria,4440,30623,735998
3,Bosnia and Herzegovina,4083,13325,288128
4,Hungary,4021,38743,1245319


#### Assigning country name as index

In [ ]:

table.index=table['country']
table.drop(table.columns[0],axis=1,inplace=True)
table.head()


,deaths_per_million,deaths,recovered
country,,,
World,686,5406818,281400646
Peru,6070,202524,2279299
Bulgaria,4440,30623,735998
Bosnia and Herzegovina,4083,13325,288128
Hungary,4021,38743,1245319


#### Checking for undefined values in columns

In [ ]:

drop_index_list=[]

for i,row in table.iterrows():
  try:
    _=int(row['deaths_per_million'])
    _=int(row['deaths'])
    _=int(row['recovered'])
    if int(row['deaths'])==0:
      print(i,' has 0 recorded deaths')
  except:
    drop_index_list.append(i)

print('The below list of countries has missing data')
drop_index_list


The below list of countries has missing data


['Falkland Islands',
 'Solomon Islands',
 'Samoa',
 'Cook Islands',
 'Marshall Islands',
 'Saint Pierre and Miquelon',
 'Palau',
 'Federated States of Micronesia',
 'Vatican City',
 'Saint Helena, Ascension and Tristan da Cunha',
 'Macau',
 'Kiribati']

We can see that the above Countries have entires with non numeric values. Hence dropping them

In [ ]:
table.drop(drop_index_list,axis=0,inplace=True)

#### Converting the columns to integer type

In [ ]:

for column in table.columns:
  table[column]=table[column].astype('int')
table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 204 entries, World to Burundi
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   deaths_per_million  204 non-null    int64
 1   deaths              204 non-null    int64
 2   recovered           204 non-null    int64
dtypes: int64(3)
memory usage: 6.4+ KB


#### Creating new column recovered_per_deaths


In [ ]:

table['recovered_per_deaths']=table['recovered']/table['deaths']
table.head()

,deaths_per_million,deaths,recovered,recovered_per_deaths
country,,,,
World,686,5406818,281400646,52.045518
Peru,6070,202524,2279299,11.254464
Bulgaria,4440,30623,735998,24.034157
Bosnia and Herzegovina,4083,13325,288128,21.623114
Hungary,4021,38743,1245319,32.143071


#### Sorting the table by recovered per deaths

In [ ]:

table=table.sort_values(by='recovered_per_deaths',ascending=False)

In [ ]:
table.head()

,deaths_per_million,deaths,recovered,recovered_per_deaths
country,,,,
Greenland,17,1,2306,2306.000000
Bhutan,3,3,2660,886.666667
Cayman Islands,165,11,8386,762.363636
Burundi,3,38,26224,690.105263
Iceland,107,37,24340,657.837838


- Higher value of the ratio indicates that more people have recovered in the country and less people had passed away due to covid. 
- Lower the ratio of recovered_per_death indicates that less people have died in the country due to Covid.
- This calculation cannot be completely relivd upon. Because there are  countries like Vanuatu, where the number of cases are very less, ie 7. 
But still they have very low value of recovered_per_death because they have total of 7 cases and 1 deaths. Hence this data needs to be further normalised

In [ ]:
table.sort_values(by='recovered_per_deaths',ascending=True).head(30)

,deaths_per_million,deaths,recovered,recovered_per_deaths
country,,,,
Yemen,65,1984,10118,5.099798
Vanuatu,3,1,7,7.000000
Peru,6070,202524,2279299,11.254464
Mexico,2293,298777,3951003,13.223920
Sudan,73,3298,46166,13.998181
Ecuador,1881,33649,542341,16.117596
Syria,157,2879,50127,17.411254
Somalia,81,1333,23532,17.653413
Egypt,207,21639,382194,17.662276
